<a href="https://colab.research.google.com/github/marquezo/darknet/blob/master/train_yolo_duckie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training an Object Detector for Duckietown
By orlando.m.bol@gmail.com

YOLO is a fast object detector with decent accuracy. It is real-time and thus useful for the Duckietown environment. For more information, see https://pjreddie.com/darknet/yolo/.

## Clone a forked version of the YOLO3 repo that contains the Duckietown datasets

In [1]:
!git clone https://github.com/marquezo/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 8134 (delta 16), reused 113 (delta 8), pack-reused 8006
Receiving objects: 100% (8134/8134), 138.94 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (4020/4020), done.


## Install CUDA and compile Darknet (which YOLO is built on)

In [2]:
!wget -O cuda_9.2.88_396.26_linux.run -c https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux

--2018-12-21 23:42:09--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?f8vS2Qi1hXZi3rKIXknSF5qvkffD76ZiO8jCMtFP_QPvrohkmiZD1fs7EtqHBx2NZPYHXf-snaYQ8aL6Oxjk3MvQjriF-LkVumv1Xug8OG9GXXhPfKcCX-Cbf63UIeTwnWOxQ6vZNclaPu1FwQPl4UwLHMuDx7ihGTzZhERt1gsN3e_UpYRHvVE [following]
--2018-12-21 23:42:10--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?f8vS2Qi1hXZi3rKIXknSF5qvkffD76ZiO8jCMtFP_QPvrohkmiZD1fs7EtqHBx2NZPYHXf-snaYQ8aL6Oxjk3MvQjriF-LkVumv1Xug8OG9GXXhPfKcCX-Cbf63UIeTwnWOxQ6vZNclaPu1FwQPl4UwLHMuDx7ihGTzZhERt1gsN3e_UpYRHvVE
Resolving developer.downl

In [3]:
!chmod +x cuda_9.2.88_396.26_linux.run
!./cuda_9.2.88_396.26_linux.run --verbose --silent --toolkit --override

Installing the CUDA Toolkit in /usr/local/cuda-9.2 ...
Verifying archive integrity... All good.

Uncompressing NVIDIA CUDA..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Include cuda in the PATH and compile Darknet**

Note that the Makefile has been modified to compile with CUDA and without OPENCV

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda-9.2/bin'

In [5]:
cd darknet

/content/darknet


In [6]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -W

## Test YOLO installation

In [7]:
# Download pre-trained weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2018-12-21 23:46:48--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.3MB/s    in 11s     

2018-12-21 23:47:00 (21.3 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [8]:
# Run object detection on sample image
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

## Train tiny YOLO on Duckietown images
We realized that the more shallow version of YOLO, named tiny YOLO, is good enough for Duckietown. Since it has fewer convolutional layers, it trains and predicts faster. You can stop the training when you like and use the saved weights to do inference.

In [9]:
# Download pre-trained weights to effectively do transfer learning
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2018-12-21 23:47:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  22.9MB/s    in 7.4s    

2018-12-21 23:47:24 (20.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
# Create a directory where we will save backups of the weights
!mkdir duckie_backup

In [11]:
# Train tiny YOLO using the already created configuration files
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector train cfg/duckie-multi.data cfg/yolov3-tiny-duckie-multi.cfg darknet53.conv.74

1984: 4.897260, 4.759357 avg, 0.001000 rate, 2.306644 seconds, 126976 images
Loaded: 0.000047 seconds
1985: 3.828319, 4.666253 avg, 0.001000 rate, 2.268940 seconds, 127040 images
Loaded: 0.000064 seconds
1986: 4.295732, 4.629201 avg, 0.001000 rate, 2.280213 seconds, 127104 images
Loaded: 0.000051 seconds
1987: 4.762844, 4.642565 avg, 0.001000 rate, 2.312915 seconds, 127168 images
Loaded: 0.000064 seconds
1988: 3.386718, 4.516981 avg, 0.001000 rate, 2.263111 seconds, 127232 images
Loaded: 0.000056 seconds
^C


## Test your Duckietown Object Detector
Pick any image from the directory `duckiestuff/testset`

In [13]:
# Let's see what has been saved during training
!ls -hlt duckie_backup/

total 332M
-rw-r--r-- 1 root root 34M Dec 22 01:20 yolov3-tiny-duckie-multi.backup
-rw-r--r-- 1 root root 34M Dec 22 00:32 yolov3-tiny-duckie-multi_900.weights
-rw-r--r-- 1 root root 34M Dec 22 00:27 yolov3-tiny-duckie-multi_800.weights
-rw-r--r-- 1 root root 34M Dec 22 00:22 yolov3-tiny-duckie-multi_700.weights
-rw-r--r-- 1 root root 34M Dec 22 00:17 yolov3-tiny-duckie-multi_600.weights
-rw-r--r-- 1 root root 34M Dec 22 00:12 yolov3-tiny-duckie-multi_500.weights
-rw-r--r-- 1 root root 34M Dec 22 00:07 yolov3-tiny-duckie-multi_400.weights
-rw-r--r-- 1 root root 34M Dec 22 00:02 yolov3-tiny-duckie-multi_300.weights
-rw-r--r-- 1 root root 34M Dec 21 23:57 yolov3-tiny-duckie-multi_200.weights
-rw-r--r-- 1 root root 34M Dec 21 23:52 yolov3-tiny-duckie-multi_100.weights


In [26]:
# Run inference
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector test cfg/duckie-multi.data cfg/yolov3-tiny-duckie-multi.cfg duckie_backup/yolov3-tiny-duckie-multi.backup duckiestuff/testset/102_000147.jpg -thresh .50

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con

## Transferring weights to Google Drive
You will want to transfer the trained weights to some other computer so that you can run inference. The easiest way is to copy them to your Google drive and then download the weights from there.

In [24]:
# Map your Google drive to Collab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Copy weights to google drive
!cp duckie_backup/yolov3-tiny-duckie-multi.backup /content/gdrive/My\ Drive/tiny-yolo-duckie.weights